### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from abc import ABC, abstractmethod
from typing import Tuple, List, Set, Dict
from lightfm.datasets import fetch_movielens

/opt/anaconda3/envs/recommendation-systems/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('./data/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('./data/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], encoding="utf16", engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [13]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [15]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '2880    Dr. No (1962)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [17]:
import random 
from utils import get_rating, get_recommendations, get_similar_items
from models.sgd_model import SGDModel
from models.als_model import ALSModel
from models.bpr_model import BPRModel
from models.warp_model import WARPModel
from base import FactorizationModel

In [18]:
def set_seed(seed: int = 42): 
    random.seed(seed)
    np.random.seed(seed)

In [19]:
id2item = movie_info[["movie_id", "name"]].set_index("movie_id").to_dict()["name"]
id2genres = movie_info[["movie_id", "category"]].set_index("movie_id").to_dict()["category"]

In [20]:
def pretty_print(candidates_ids: List[int]): 
    for i, candidate_id in enumerate(candidates_ids, start=1): 
        print(f"{i}. {id2item[candidate_id]} ({id2genres[candidate_id]})")

In [21]:
def print_similars(item_id: int, model: FactorizationModel):
    similar_items_idx = get_similar_items(item_id, model, top_k=10)
    print(f"Similar items for '{id2item[item_id]}' ({id2genres[item_id]}):")
    pretty_print(similar_items_idx)

In [22]:
def print_recommendations(user_id: int, model: FactorizationModel): 
    recommend_items_idx = get_recommendations(user_id, model, top_k=10)
    print(f"Recommendations for '{user_id}':")
    pretty_print(recommend_items_idx)

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

Создадим explicit набор данных и обучим модель. 

In [23]:
set_seed()
user_item_explicit = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"]))).tocsr()
sgd_model = SGDModel(factors=64, lr=1e-2, iterations=50, verbose=True, verbose_every=10)
sgd_model = sgd_model.fit(user_item_explicit)

 20%|██        | 10/50 [01:31<05:55,  8.89s/it]

Iter: 10 | RMSE: 0.68 | Elapsed: 1m32s


 40%|████      | 20/50 [03:01<04:30,  9.01s/it]

Iter: 20 | RMSE: 0.58 | Elapsed: 3m1s


 60%|██████    | 30/50 [04:27<02:52,  8.63s/it]

Iter: 30 | RMSE: 0.54 | Elapsed: 4m27s


 80%|████████  | 40/50 [05:53<01:25,  8.59s/it]

Iter: 40 | RMSE: 0.53 | Elapsed: 5m53s


100%|██████████| 50/50 [07:18<00:00,  8.78s/it]

Iter: 50 | RMSE: 0.53 | Elapsed: 7m19s


Проверим качество обученной модели.

In [25]:
print_similars(1, sgd_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Toy Story 2 (1999) (Animation|Children's|Comedy)
3. Bug's Life, A (1998) (Animation|Children's|Comedy)
4. Wrong Trousers, The (1993) (Animation|Comedy)
5. Close Shave, A (1995) (Animation|Comedy|Thriller)
6. Aladdin (1992) (Animation|Children's|Comedy|Musical)
7. Back to the Future (1985) (Comedy|Sci-Fi)
8. Grand Day Out, A (1992) (Animation|Comedy)
9. My Man Godfrey (1936) (Comedy)
10. Shawshank Redemption, The (1994) (Drama)


Как можно видеть, фильмы достаточно подходящие.

In [26]:
print_recommendations(4, sgd_model)

Recommendations for '4':
1. Casablanca (1942) (Drama|Romance|War)
2. Lawrence of Arabia (1962) (Adventure|War)
3. Shadow of a Doubt (1943) (Film-Noir|Thriller)
4. Face/Off (1997) (Action|Sci-Fi|Thriller)
5. Animal House (1978) (Comedy)
6. Bicycle Thief, The (Ladri di biciclette) (1948) (Drama)
7. Mr. Holland's Opus (1995) (Drama)
8. Silence of the Lambs, The (1991) (Drama|Thriller)
9. Boat, The (Das Boot) (1981) (Action|Drama|War)
10. It's a Wonderful Life (1946) (Drama)


Фантастики и боевиков получилось мало :(

Видимо, придется к хард-дедлайну делать SGD со смещениями и регуляризацией и мини-батчами...

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [27]:
set_seed()
user_item_implicit = user_item_csr
als_model = ALSModel(factors=64, iterations=10, lambd=1e-5, verbose=True, verbose_every=1)
als_model = als_model.fit(user_item_implicit)

 10%|█         | 1/10 [00:12<01:50, 12.25s/it]

Iter: 1 | RMSE: 0.73 | Elapsed: 0m12s


 20%|██        | 2/10 [00:26<01:49, 13.65s/it]

Iter: 2 | RMSE: 0.65 | Elapsed: 0m26s


 30%|███       | 3/10 [00:40<01:36, 13.77s/it]

...

Iter: 8 | RMSE: 0.64 | Elapsed: 1m45s


 90%|█████████ | 9/10 [01:58<00:12, 12.97s/it]

Iter: 9 | RMSE: 0.64 | Elapsed: 1m58s


100%|██████████| 10/10 [02:11<00:00, 13.20s/it]

Iter: 10 | RMSE: 0.64 | Elapsed: 2m12s


Проверим качество обученное модели. 

In [28]:
print_similars(1, als_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Toy Story 2 (1999) (Animation|Children's|Comedy)
3. Bug's Life, A (1998) (Animation|Children's|Comedy)
4. Babe (1995) (Children's|Comedy|Drama)
5. Aladdin (1992) (Animation|Children's|Comedy|Musical)
6. Hercules (1997) (Adventure|Animation|Children's|Comedy|Musical)
7. Mulan (1998) (Animation|Children's)
8. Babe: Pig in the City (1998) (Children's|Comedy)
9. Lion King, The (1994) (Animation|Children's|Musical)
10. Tarzan (1999) (Animation|Children's)


Почти все фильмы оказались подходящими!

In [29]:
print_recommendations(4, als_model)

Recommendations for '4':
1. Raiders of the Lost Ark (1981) (Action|Adventure)
2. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
3. Jaws (1975) (Action|Horror)
4. Terminator, The (1984) (Action|Sci-Fi|Thriller)
5. Saving Private Ryan (1998) (Action|Drama|War)
6. E.T. the Extra-Terrestrial (1982) (Children's|Drama|Fantasy|Sci-Fi)
7. Alien (1979) (Action|Horror|Sci-Fi|Thriller)
8. Die Hard (1988) (Action|Thriller)
9. Terminator 2: Judgment Day (1991) (Action|Sci-Fi|Thriller)
10. Goldfinger (1964) (Action)


Вот, здесь все норм. 

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [30]:
from models.bpr_model import BPRModel

In [31]:
set_seed()
bpr_model = BPRModel(factors=64, lr=1e-2, iterations=100, lambd=1e-6, verbose=True, verbose_every=10)
bpr_model = bpr_model.fit(user_item_implicit)

 10%|█         | 10/100 [00:15<02:23,  1.60s/it]

Iter: 10 | Triplet acc: 0.84 | Elapsed: 0m15s


 20%|██        | 20/100 [00:30<01:54,  1.43s/it]

Iter: 20 | Triplet acc: 0.87 | Elapsed: 0m30s


 30%|███       | 30/100 [00:45<01:51,  1.59s/it]

...

Iter: 80 | Triplet acc: 0.89 | Elapsed: 2m0s


 90%|█████████ | 90/100 [02:15<00:14,  1.41s/it]

Iter: 90 | Triplet acc: 0.89 | Elapsed: 2m15s


100%|██████████| 100/100 [02:29<00:00,  1.49s/it]

Iter: 100 | Triplet acc: 0.89 | Elapsed: 2m29s


In [32]:
print_similars(1, bpr_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Groundhog Day (1993) (Comedy|Romance)
3. Back to the Future (1985) (Comedy|Sci-Fi)
4. American Beauty (1999) (Comedy|Drama)
5. Saving Private Ryan (1998) (Action|Drama|War)
6. Braveheart (1995) (Action|Drama|War)
7. Sixth Sense, The (1999) (Thriller)
8. Silence of the Lambs, The (1991) (Drama|Thriller)
9. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
10. Star Wars: Episode VI - Return of the Jedi (1983) (Action|Adventure|Romance|Sci-Fi|War)


Определенно покажу своему сыну в детстве "Красоту по-американски" и "Молчание ягнят". 
Мои эмоции: https://www.youtube.com/watch?v=pTZaNHZGsQo

In [33]:
print_recommendations(4, bpr_model)

Recommendations for '4':
1. American Beauty (1999) (Comedy|Drama)
2. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
3. Saving Private Ryan (1998) (Action|Drama|War)
4. Silence of the Lambs, The (1991) (Drama|Thriller)
5. Star Wars: Episode V - The Empire Strikes Back (1980) (Action|Adventure|Drama|Sci-Fi|War)
6. Sixth Sense, The (1999) (Thriller)
7. Star Wars: Episode VI - Return of the Jedi (1983) (Action|Adventure|Romance|Sci-Fi|War)
8. Schindler's List (1993) (Drama|War)
9. Raiders of the Lost Ark (1981) (Action|Adventure)
10. Braveheart (1995) (Action|Drama|War)


А тут все отлично! Продолжения фильмов, все круто!

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [34]:
set_seed()
warp_model = WARPModel(factors=64, lr=1e-2, iterations=35, verbose=True, verbose_every=5)
warp_model = warp_model.fit(user_item_implicit)

  3%|▎         | 1/35 [00:01<01:06,  1.97s/it]

Iter: 1 | Triplet acc: 0.78 | Elapsed: 0m1s


 17%|█▋        | 6/35 [00:20<01:57,  4.04s/it]

Iter: 6 | Triplet acc: 0.99 | Elapsed: 0m20s


 31%|███▏      | 11/35 [00:55<02:39,  6.66s/it]

...

Iter: 26 | Triplet acc: 1.00 | Elapsed: 3m24s


 89%|████████▊ | 31/35 [05:32<01:39, 24.82s/it]

Iter: 31 | Triplet acc: 1.00 | Elapsed: 5m32s


100%|██████████| 35/35 [07:51<00:00, 13.46s/it]

Iter: 36 | Triplet acc: 1.00 | Elapsed: 7m51s


In [35]:
print_similars(1, warp_model)

Similar items for 'Toy Story (1995)' (Animation|Children's|Comedy):
1. Toy Story (1995) (Animation|Children's|Comedy)
2. Snow White and the Seven Dwarfs (1937) (Animation|Children's|Musical)
3. Grand Day Out, A (1992) (Animation|Comedy)
4. Wayne's World (1992) (Comedy)
5. First Wives Club, The (1996) (Comedy)
6. Evil Dead II (Dead By Dawn) (1987) (Action|Adventure|Comedy|Horror)
7. Mary Poppins (1964) (Children's|Comedy|Musical)
8. Bottle Rocket (1996) (Comedy)
9. Diner (1982) (Comedy|Drama)
10. Children of Heaven, The (Bacheha-Ye Aseman) (1997) (Drama)


In [36]:
print_recommendations(4, warp_model)

Recommendations for '4':
1. Raiders of the Lost Ark (1981) (Action|Adventure)
2. Terminator, The (1984) (Action|Sci-Fi|Thriller)
3. Alien (1979) (Action|Horror|Sci-Fi|Thriller)
4. Die Hard (1988) (Action|Thriller)
5. Star Wars: Episode IV - A New Hope (1977) (Action|Adventure|Fantasy|Sci-Fi)
6. Jurassic Park (1993) (Action|Adventure|Sci-Fi)
7. E.T. the Extra-Terrestrial (1982) (Children's|Drama|Fantasy|Sci-Fi)
8. Men in Black (1997) (Action|Adventure|Comedy|Sci-Fi)
9. Total Recall (1990) (Action|Adventure|Sci-Fi|Thriller)
10. Jaws (1975) (Action|Horror)


Рекомендации крутые, а вот похожести странные. Возможно, это особенности алгоритмов. 